<a href="https://colab.research.google.com/github/chinmay002/LLM-s/blob/main/Salesforce_LLMa2_QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.32.1 --progress-bar off
!pip install -qqq datasets==2.14.4 --progress-bar off
!pip install -qqq peft==0.5.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install -qqq trl==0.7.1 --progress-bar off


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.


In [3]:
import json
import torch
import pandas as pd
import numpy as np

from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,TrainingArguments,Trainer,BitsAndBytesConfig
from datasets import Dataset,load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig,PeftModel

device = 'cuda:0'
model_name = "meta-llama/Llama-2-7b-hf"

#Data

In [4]:
dataset = load_dataset("Salesforce/dialogstudio", "TweetSumm")
dataset

DatasetDict({
    train: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 879
    })
    validation: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
    test: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
})

In [5]:
res = json.loads(dataset['train'].to_pandas()['original dialog info'][0])
res

{'summaries': {'extractive_summaries': [[{'is_agent': False,
     'sentences': ['So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason.']},
    {'is_agent': True,
     'sentences': ['To start, can you tell us the software versions your iPhone and Apple Watch are running currently?']},
    {'is_agent': False,
     'sentences': ['@AppleSupport My iPhone is on 11.1.2, and my watch is on 4.1.']}],
   [{'is_agent': False,
     'sentences': ['So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason.']},
    {'is_agent': True,
     'sentences': ['To start, can you tell us the software versions your iPhone and Apple Watch are running currently?']}],
   [{'is_agent': False,
     'sentences': ['So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reas

In [ ]:
summaries = json.loads(dataset["train"][0]["original dialog info"])["summaries"]["abstractive_summaries"]
summary = summaries[0]
summary = " ".join(summary)
summary

'Customer enquired about his Iphone and Apple watch which is not showing his any steps/activity and health activities. Agent is asking to move to DM and look into it.'

In [ ]:
for item in dataset["train"][0]['log']:
  print(item)

{'turn id': 1, 'user utterance': 'So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? https://t.co/m9DPQbkftD @AppleSupport please read the above.', 'system response': '@135060 Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?', 'dialog history': '', 'original user side information': '{}', 'original system side information': '{}'}
{'turn id': 2, 'user utterance': '@AppleSupport My iPhone is on 11.1.2, and my watch is on 4.1.', 'system response': '@135060 Thank you. Have you tried restarting both devices since this started happening?', 'dialog history': '<USER> So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? https://t.co/m9DPQbkftD @AppleSupport please read the above. <SYSTEM> @135060 Let’s investigate thi

In [ ]:
text = ''
for item in dataset["train"][0]['log']:
    text += f"user :  {item['user utterance']}\n"
    text += f"agent :  {item['system response']}\n"

print(text)

user :  So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? https://t.co/m9DPQbkftD @AppleSupport please read the above.
agent :  @135060 Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?
user :  @AppleSupport My iPhone is on 11.1.2, and my watch is on 4.1.
agent :  @135060 Thank you. Have you tried restarting both devices since this started happening?
user :  @AppleSupport I’ve restarted both, also un-paired then re-paired the watch.
agent :  @135060 Got it. When did you first notice that the two devices were not talking to each other. Do the two devices communicate through other apps such as Messages?
user :  @AppleSupport Yes, everything seems fine, it’s just Health and activity.
agent :  @135060 Let’s move to DM and look into this a bit more. When reaching out in DM, let us know when this first started 

In [ ]:
#lets clean the text
import re
def clean_text(text):
  text = re.sub(r"http\S+", "", text)
  text = re.sub(r"@[^\s]+", "", text)
  text = re.sub(r"\s+", " ", text)
  return re.sub(r"\^[^ ]+", "", text)

In [ ]:
text = ''
for item in dataset["train"][0]['log']:
    text += f"user : {clean_text(item['user utterance'].strip())}\n"
    text += f"agent : {clean_text(item['system response']).strip()}\n"

print(text)

user : So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? please read the above.
agent : Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?
user :  My iPhone is on 11.1.2, and my watch is on 4.1.
agent : Thank you. Have you tried restarting both devices since this started happening?
user :  I’ve restarted both, also un-paired then re-paired the watch.
agent : Got it. When did you first notice that the two devices were not talking to each other. Do the two devices communicate through other apps such as Messages?
user :  Yes, everything seems fine, it’s just Health and activity.
agent : Let’s move to DM and look into this a bit more. When reaching out in DM, let us know when this first started happening please. For example, did it start after an update or after installing a certain app?



In [5]:
import re
def clean_text(text):
  text = re.sub(r"http\S+", "", text)
  text = re.sub(r"@[^\s]+", "", text)
  text = re.sub(r"\s+", " ", text)
  return re.sub(r"\^[^ ]+", "", text)


def create_conversation_text(data_point):
  text = ''
  for item in data_point['log']:
    text += f"user :  {clean_text(item['user utterance'].strip())}\n"
    text += f"agent :  {clean_text(item['system response']).strip()}\n"
  return text



default_prompt = """
Below is the conversation between a human and AI agent, write a summary if the conversation
""".strip()


def generate_training_prompt(conversation,summary,sys_prompt = default_prompt):
  return  f"""### INSTRUCTION :
  {sys_prompt}

  ###INPUT :
  {conversation.strip()}

  ###Response :
  {summary.strip()}

  """




In [6]:
def generate_data(data_point):
  summaries = json.loads(data_point["original dialog info"])["summaries"][
        "abstractive_summaries"
    ]
  summary = summaries[0]
  summary = " ".join(summary)

  conversation_text = create_conversation_text(data_point)
  return {
      'conversation':conversation_text,
      'summary' : summary,
      'text' : generate_training_prompt(conversation_text,summary)
  }


In [6]:
example = generate_data(dataset["train"][0])
print(example['summary'])
print()
print(example['conversation'])

Customer enquired about his Iphone and Apple watch which is not showing his any steps/activity and health activities. Agent is asking to move to DM and look into it.

user :  So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? please read the above.
agent :  Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?
user :   My iPhone is on 11.1.2, and my watch is on 4.1.
agent :  Thank you. Have you tried restarting both devices since this started happening?
user :   I’ve restarted both, also un-paired then re-paired the watch.
agent :  Got it. When did you first notice that the two devices were not talking to each other. Do the two devices communicate through other apps such as Messages?
user :   Yes, everything seems fine, it’s just Health and activity.
agent :  Let’s move to DM and look into this a bit more. Whe

In [ ]:
print(example['text'])

### INSTRUCTION : 
  Below is the conversation between a human and AI agent, write a summary if the conversation  
  
  ###INPUT : 
  user :  So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? please read the above.
agent :  Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?
user :   My iPhone is on 11.1.2, and my watch is on 4.1.
agent :  Thank you. Have you tried restarting both devices since this started happening?
user :   I’ve restarted both, also un-paired then re-paired the watch.
agent :  Got it. When did you first notice that the two devices were not talking to each other. Do the two devices communicate through other apps such as Messages?
user :   Yes, everything seems fine, it’s just Health and activity.
agent :  Let’s move to DM and look into this a bit more. When reaching out in DM, let us know

In [7]:
def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_data)
        .remove_columns(
            [
                "original dialog id",
                "new dialog id",
                "dialog index",
                "original dialog info",
                "log",
                "prompt",
            ]
        )
    )

In [19]:
dataset['train']

Dataset({
    features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
    num_rows: 879
})

In [8]:
dataset['train'] = process_dataset(dataset['train'].select(range(300)))
dataset['validation'] = process_dataset(dataset['validation'])

In [20]:
dataset['train'].select(range(300))

Dataset({
    features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
    num_rows: 100
})

##Model

In [9]:
notebook_login()

In [10]:

def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer


model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [23]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [11]:

lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

#Training

In [14]:

OUTPUT_DIR = "experiments"

%load_ext tensorboard
%tensorboard --logdir experiments/runs

In [15]:

training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
)


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [16]:
trainer.train()


TypeError: ignored

In [17]:

trainer.save_model()

In [22]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
              

In [19]:
from peft import AutoPeftModelForCausalLM

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    low_cpu_mem_usage=True,
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.

In [23]:
merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged_model", safe_serialization=True)
tokenizer.save_pretrained("merged_model")


AttributeError: ignored